In [1]:
from prepare_jpype import start_jpype
import pandas as pd
start_jpype()

# local code imports
from evaluation import *
from ensembles import OnlineBagging
from stream import *

arff_elec_path = '/Users/gomeshe/Desktop/data/electricity.arff'
csv_elec_path = '/Users/gomeshe/Desktop/data/electricity.csv'

MOA jar path location (config.ini): /Users/gomeshe/Dropbox/ciencia_computacao/dev/Using-MOA-API/moa.jar
JVM Location (system): 
/Users/gomeshe/Library/Java/JavaVirtualMachines/openjdk-20.0.1/Contents/Home
Sucessfully started the JVM and added MOA jar to the class path


## Testing Data Reading, including a method to transform from numpy to ARFF

In [2]:
maxInstancesToProcess = 5000
sampleFrequency = 1000
instancesProcessed = 1

stream = ARFFStream(path=arff_elec_path)
# stream.get_schema().get_label_values() is None

learner = OnlineBagging(schema=stream.get_schema(), ensemble_size=5)

evaluator_TTT = ClassificationEvaluator(schema=stream.get_schema(), window_size=sampleFrequency)
evaluator_windowed = ClassificationWindowedEvaluator(schema=stream.get_schema(), window_size=sampleFrequency)

while stream.has_more_instances() and instancesProcessed <= maxInstancesToProcess:
    instance = stream.next_instance()
    prediction = learner.predict(instance)
    evaluator_TTT.update(instance.getData().classValue(),prediction)
    evaluator_windowed.update(instance.getData().classValue(),prediction)
    learner.train(instance)
    
    instancesProcessed += 1

print(evaluator_TTT.accuracy())
evaluator_windowed.metrics_per_window()

84.48


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,1000.0,85.7,71.415778,-1.418440,70.816327
1,2000.0,83.5,61.565339,-17.857143,44.630872
2,3000.0,80.8,60.255853,-3.225806,53.284672
3,4000.0,87.6,73.842863,26.627219,68.286445
4,5000.0,84.8,64.548270,5.000000,57.062147


## Testing the ```numpy_to_ARFF()``` function

In [3]:
# Import necessary libraries
from sklearn.datasets import load_iris

# Load the Iris dataset
data = load_iris()
X = data.data  # Features
y = data.target  # Target labels

print(numpy_to_ARFF(X, y, "Iris"))

(<java object 'com.yahoo.labs.samoa.instances.Instances'>, <java object 'com.yahoo.labs.samoa.instances.InstancesHeader'>)


In [4]:
# Import necessary libraries
from sklearn.datasets import load_diabetes

# Load the Diabetes dataset
data = load_diabetes()
X = data.data  # Features
y = data.target  # Target (diabetes progression)

# print(data.DESCR)
# print(numpy_to_ARFF(X, y, "Diabetes"))

arff_instances_data, arff_instances_header = numpy_to_ARFF(X, y, "Diabetes", data.feature_names)

print(arff_instances_header)
print(arff_instances_data)

@relation Diabetes

@attribute age numeric
@attribute sex numeric
@attribute bmi numeric
@attribute bp numeric
@attribute s1 numeric
@attribute s2 numeric
@attribute s3 numeric
@attribute s4 numeric
@attribute s5 numeric
@attribute s6 numeric
@attribute target numeric

@data

@relation Diabetes

@attribute age numeric
@attribute sex numeric
@attribute bmi numeric
@attribute bp numeric
@attribute s1 numeric
@attribute s2 numeric
@attribute s3 numeric
@attribute s4 numeric
@attribute s5 numeric
@attribute s6 numeric
@attribute target numeric

@data
0.038075906433423026,0.05068011873981862,0.061696206518683294,0.0218723855140367,-0.04422349842444599,-0.03482076283769895,-0.04340084565202491,-0.002592261998183278,0.019907486170462722,-0.01764612515980379,151.0,
-0.0018820165277906047,-0.044641636506989144,-0.051474061238800654,-0.02632752814785296,-0.008448724111216851,-0.019163339748222204,0.07441156407875721,-0.03949338287409329,-0.0683315470939731,-0.092204049626824,75.0,
0.085298906296

## Example reading the csv from a file using ```np.genfromtxt```

In [5]:
data = np.genfromtxt(csv_elec_path, delimiter=',', skip_header=1)  # Assuming a header row

# Extract the feature data (all columns except the last one) and target data (last column)
X = data[:, :-1]
y = data[:, -1]

# Extract the header from the CSV file (first row)
with open(csv_elec_path, 'r') as file:
    header = file.readline().strip().split(',')

# Optionally, you can print the shapes of X and y to verify
print("X shape:", X.shape)
print("y shape:", y.shape)
print(header)

arff_data, arff_header = numpy_to_ARFF(X, y.astype(int), "Elec", header[:-1], header[-1])

# schema=Schema(moa_header=arff_data.getHeader())
print(arff_header)

X shape: (45312, 6)
y shape: (45312,)
['period', 'nswprice', 'nswdemand', 'vicprice', 'vicdemand', 'transfer', 'class']
@relation Elec

@attribute period numeric
@attribute nswprice numeric
@attribute nswdemand numeric
@attribute vicprice numeric
@attribute vicdemand numeric
@attribute transfer numeric
@attribute class {0,1}

@data



## Example: Creating a stream using X and y (NumpyStream) 

## Testing every method of the NumpyStream class

In [6]:
data = np.genfromtxt(csv_elec_path, delimiter=',', skip_header=1)  # Assuming a header row

# Extract the feature data (all columns except the last one) and target data (last column)
X = data[:, :-1]
y = data[:, -1]

# Extract the header from the CSV file (first row)
with open(csv_elec_path, 'r') as file:
    header = file.readline().strip().split(',')
    
np_stream = NumpyStream(X=X, y=y.astype(int), dataset_name="Elec", feature_names=header[:-1], target_name=header[-1])


print(np_stream.next_instance())
print(np_stream.next_instance())
print(np_stream.next_instance())

print("restarting stream...")

np_stream.restart()

print(np_stream.next_instance())
print(np_stream.next_instance())
print(np_stream.next_instance())

np_stream.restart()

try:
    np_stream.get_moa_stream()
except ValueError as ve:
    print(ve)

counter_num_instances = 0
while np_stream.has_more_instances():
    np_stream.next_instance()
    counter_num_instances+=1

np_stream.next_instance()

print(f"Read {counter_num_instances} instances, total = {np_stream.arff_instances_data.numInstances() }")

0.0,0.056443,0.439155,0.003467,0.422915,0.414912,1,
0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,1,
0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,1,
restarting stream...
0.0,0.056443,0.439155,0.003467,0.422915,0.414912,1,
0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,1,
0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,1,
Not a moa_stream, a numpy read file
Read 45312 instances, total = 45312


## Using ```NumpyStream``` to create the stream and training an OnlineBagging object to be trained on it.

In [7]:
data = np.genfromtxt(csv_elec_path, delimiter=',', skip_header=1)  # Assuming a header row

# Extract the feature data (all columns except the last one) and target data (last column)
X = data[:, :-1]
y = data[:, -1]

# Extract the header from the CSV file (first row)
with open(csv_elec_path, 'r') as file:
    header = file.readline().strip().split(',')
    
np_stream = NumpyStream(X=X, y=y.astype(int), dataset_name="Elec", feature_names=header[:-1], target_name=header[-1])

ob_learner = OnlineBagging(schema=np_stream.get_schema(), ensemble_size=5)

ob_windowed_evaluator = ClassificationWindowedEvaluator(schema=np_stream.get_schema(), window_size=5000)

while np_stream.has_more_instances():
    instance = np_stream.next_instance()
    
    prediction = ob_learner.predict(instance)
    ob_windowed_evaluator.update(instance.getData().classValue(), prediction)
    ob_learner.train(instance)

ob_windowed_evaluator.metrics_per_window()

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,5000.0,84.48,67.192489,2.512563,60.082305
1,10000.0,82.92,65.723322,-3.015682,64.192872
2,15000.0,84.26,68.414479,-11.948791,66.581741
3,20000.0,78.48,52.881349,-49.237171,47.716229
4,25000.0,73.98,40.318363,-65.732484,32.168926
5,30000.0,76.78,49.165527,-106.583630,43.060324
6,35000.0,73.18,43.235885,-98.961424,34.871297
7,40000.0,74.80,50.791615,-71.428571,37.561943
8,45000.0,82.60,65.046029,-10.687023,63.628763


## Testing ```stream = stream_from_file(path_to_csv_or_arff=...)```

In [8]:
stream = stream_from_file(path_to_csv_or_arff=arff_elec_path)

print(stream.next_instance())

stream.get_moa_stream()

0.0,0.056443,0.439155,0.003467,0.422915,0.414912,1,


<java object 'moa.streams.ArffFileStream'>

In [9]:
stream = stream_from_file(path_to_csv_or_arff=csv_elec_path)

print(stream.next_instance())

## This method will raise an error as expected! 
# Furthermore, the user is not expected to invoke the get_moa_stream directly, unless they know what they are doing. 
# stream.get_moa_stream()

0.0,0.056443,0.439155,0.003467,0.422915,0.414912,1,


## Using ```stream_from_file``` using ARFF and using CSV (should output same results)

Using 200 learners in Online Bagging to confirm that after reading the file there is no difference in runtime. 

In [10]:
%%time
stream = stream_from_file(path_to_csv_or_arff=arff_elec_path)

ob_learner = OnlineBagging(schema=stream.get_schema(), ensemble_size=200)

ob_windowed_evaluator = ClassificationWindowedEvaluator(schema=stream.get_schema(), window_size=5000)

while stream.has_more_instances():
    instance = stream.next_instance()
    
    prediction = ob_learner.predict(instance)
    ob_windowed_evaluator.update(instance.getData().classValue(), prediction)
    ob_learner.train(instance)

ob_windowed_evaluator.metrics_per_window()

CPU times: user 19.4 s, sys: 59.4 ms, total: 19.4 s
Wall time: 18.7 s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,5000.0,84.52,67.117427,2.763819,60.185185
1,10000.0,83.52,66.946949,0.603136,65.450734
2,15000.0,83.60,67.061685,-16.642959,65.180467
3,20000.0,78.72,53.312857,-47.572816,48.299320
4,25000.0,74.72,41.585243,-61.019108,34.098019
5,30000.0,77.20,49.592558,-102.846975,44.090240
6,35000.0,74.82,46.314066,-86.795252,38.853813
7,40000.0,74.28,50.121725,-74.965986,36.273538
8,45000.0,82.62,65.033862,-10.559796,63.670569


In [11]:
%%time
stream = stream_from_file(path_to_csv_or_arff=csv_elec_path)

ob_learner = OnlineBagging(schema=stream.get_schema(), ensemble_size=200)

ob_windowed_evaluator = ClassificationWindowedEvaluator(schema=stream.get_schema(), window_size=5000)

while stream.has_more_instances():
    instance = stream.next_instance()
    
    prediction = ob_learner.predict(instance)
    ob_windowed_evaluator.update(instance.getData().classValue(), prediction)
    ob_learner.train(instance)

ob_windowed_evaluator.metrics_per_window()

CPU times: user 19.6 s, sys: 56 ms, total: 19.7 s
Wall time: 19.2 s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,5000.0,84.52,67.117427,2.763819,60.185185
1,10000.0,83.52,66.946949,0.603136,65.450734
2,15000.0,83.60,67.061685,-16.642959,65.180467
3,20000.0,78.72,53.312857,-47.572816,48.299320
4,25000.0,74.72,41.585243,-61.019108,34.098019
5,30000.0,77.20,49.592558,-102.846975,44.090240
6,35000.0,74.82,46.314066,-86.795252,38.853813
7,40000.0,74.28,50.121725,-74.965986,36.273538
8,45000.0,82.62,65.033862,-10.559796,63.670569
